In [1]:
import numpy as np
import pandas as pd

In [18]:
df = pd.ExcelFile('Retirement_Services_L8_initiative.xlsx')

In [23]:
df.parse(df.sheet_names[0],header=None)

,0,1,2,3,4,5,6,7,8,9,...,32,33,34,35,36,37,38,39,40,41
0,Managed Segment:,Retirement Services [L8],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Managed Geography:,Total Mexico,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,Year,2020.0,2020.0,2020.0,2020.0,...,2022.0,2022.0,2022.0,2022.0,2022.0,2022.0,2022.0,2022.0,2022.0,2022.0
3,NaN,NaN,NaN,NaN,NaN,Qtr,1.0,1.0,1.0,2.0,...,1.0,2.0,2.0,2.0,3.0,3.0,3.0,4.0,4.0,4.0
4,BD_ID,BD_NAME,CURRENCY,Action Type,Weighted Node ID,Month,1.0,2.0,3.0,4.0,...,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,11.0,12.0
5,BD_00260025,Gains / losses volume - Adjustment,MXN,I,,NaN,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,BD_00260025,Gains / losses volume - Adjustment,USD,I,,NaN,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,BD_00260038,Allocations - Other Liabilities,MXN,I,,NaN,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,BD_00260038,Allocations - Other Liabilities,USD,I,,NaN,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,BD_00260057,Allocation - FX,MXN,I,,NaN,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [38]:
data = df.parse(df.sheet_names[0],header=None)

In [56]:
data[data[3] == 'W'][5].isnull()

19    True
20    True
25    True
26    True
31    True
32    True
39    True
40    True
45    True
46    True
49    True
50    True
Name: 5, dtype: bool

In [99]:
import pandas as pd
import numpy as np

In [155]:
xls = pd.ExcelFile('template.xlsx')

In [156]:
[xls.parse(xls.sheet_names[i]) for i in range(xls.sheet_names.__len__())]

[            BD_ID                             BD_NAME CURRENCY Action Type  \
 0     BD_00260025  Gains / losses volume - Adjustment      MXN           W   
 1     BD_00260025  Gains / losses volume - Adjustment      MXN         NaN   
 2     BD_00260025  Gains / losses volume - Adjustment      MXN           W   
 3     BD_00260025  Gains / losses volume - Adjustment      MXN           W   
 4     BD_00260025  Gains / losses volume - Adjustment      MXN           W   
 5     BD_00260025  Gains / losses volume - Adjustment      MXN           W   
 6     BD_00260025  Gains / losses volume - Adjustment      MXN         NaN   
 7     BD_00260025  Gains / losses volume - Adjustment      MXN           W   
 8     BD_00260025  Gains / losses volume - Adjustment      MXN           W   
 9     BD_00260025  Gains / losses volume - Adjustment      MXN           W   
 10    BD_00260025  Gains / losses volume - Adjustment      MXN           W   
 11    BD_00260025  Gains / losses volume - Adjustme

In [206]:
validation_errors = []
for i in range(xls.sheet_names.__len__()):
    df = xls.parse(xls.sheet_names[i])
    
    #5. Validate that for all nodes whose Action Type is W should have Weighted Node ID for that node.
    if not df[df["Action Type"]=='W']['Weighted Node ID'].str.contains(' ',na=False).index[df[df["Action Type"]=='W']['Weighted Node ID'].str.contains(' ',na=False)].__len__() == 0:
        null_weighted_bd_ids = df.iloc[df[df["Action Type"]=='W']['Weighted Node ID'].str.contains(' ',na=False).index[df[df["Action Type"]=='W']['Weighted Node ID'].str.contains(' ',na=False)]]["BD_ID"].unique()
        for idx, bd_id in enumerate(null_weighted_bd_ids):
            validation_errors.append(f"Weighted Node Id against {bd_id} for {xls.sheet_names[i]} is blank.")
    
    # 8.  Weighted Node ID should follow the correct format i.e. BD_XXXXXXXX or C_XXXXXXXX, or R_XXXXXXXX
    li = [df['BD_ID'].str.startswith(i, na=False) for i in ["BD_","C_","R_"]]
    if not df.__len__() - np.sum(np.column_stack((li[0],li[1],li[2])).any(axis=1)) == 0:
        temp = pd.Series(np.column_stack((li[0],li[1],li[2])).any(axis=1))
        incorrect_bd_ids = df.iloc[temp[temp == False].index]["BD_ID"]
        for idx, bd_id in enumerate(incorrect_bd_ids):
            validation_errors.append(f"{bd_id} in {xls.sheet_names[i]} doesn't follow the correct format. ")
    
    # 12. Action Type value is other than I, R or W
    if not df["Action Type"].apply(lambda x: True if x not in ["I","R","W"] else False).sum() == 0:
        unknown_action_type = df[-df["Action Type"].isin(["I","R","W"])]
        unknown_action_bd_ids = unknown_action_type.xs("BD_ID",axis=1).unique()
        for idx, bd_id in enumerate(unknown_action_bd_ids):
            validation_errors.append(f"{bd_id} in {xls.sheet_names[i]} have Action Type other than I, R or W.")
            
    # 13. Action Type value should not be blank
    if df["Action Type"].isnull().any():
        null_action_index = df[df["Action Type"].isna()].index
        null_action_bd_ids = df.iloc[null_action_index,:]["BD_ID"].unique()
        for idx, bd_id in enumerate(null_action_bd_ids):
            validation_errors.append(f"Action Type for {bd_id} in {xls.sheet_names[i]} cannot be blank. ")
            
    # 7. R Action Type should have non zero values.
    if not df[df["Action Type"]=='R']["Values"].value_counts()[0] == 0:
        r_action_null_values_bd_ids = df.iloc[(df[df["Action Type"]=='R']["Values"]==0).index[(df[df["Action Type"]=='R']["Values"]==0)]]["BD_ID"].unique()
        for idx, bd_ids in enumerate(r_action_null_values_bd_ids):
            validation_errors.append(f"{bd_ids} is R action type and must have a value in {xls.sheet_names[i]}.")

In [207]:
[print(i) for i in validation_errors]

Weighted Node Id against BD_00260025 for Sheet1 is blank.
Weighted Node Id against BD_00260032 for Sheet1 is blank.
Weighted Node Id against R_00260004 for Sheet1 is blank.
Weighted Node Id against R_00260005 for Sheet1 is blank.
Weighted Node Id against R_00260001 for Sheet1 is blank.
Weighted Node Id against R_00260003 for Sheet1 is blank.
Weighted Node Id against R_00260002 for Sheet1 is blank.
Weighted Node Id against 2002 for Sheet1 is blank.
Weighted Node Id against 2 for Sheet1 is blank.
XYZ in Sheet1 doesn't follow the correct format. 
2002 in Sheet1 doesn't follow the correct format. 
2 in Sheet1 doesn't follow the correct format. 
BD_00260025 in Sheet1 have Action Type other than I, R or W.
Action Type for BD_00260025 in Sheet1 cannot be blank. 
BD_00260025 is R action type and must have a value in Sheet1.
XYZ is R action type and must have a value in Sheet1.
BD_00260019 is R action type and must have a value in Sheet1.


[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [180]:
df[df["Action Type"]=='W']['Weighted Node ID'].str.contains(' ',na=False).index[df[df["Action Type"]=='W']['Weighted Node ID'].str.contains(' ',na=False)].__len__()

464

In [199]:
#df.iloc[(df[df["Action Type"]=='R']["Values"]==0).index[(df[df["Action Type"]=='R']["Values"]==0)]]["BD_ID"]

### R action type should have non zero value

In [209]:
if not df[df["Action Type"]=='R']["Values"].value_counts()[0] == 0:
    r_action_null_values_bd_ids = df.iloc[(df[df["Action Type"]=='R']["Values"]==0).index[(df[df["Action Type"]=='R']["Values"]==0)]]["BD_ID"].unique()
    for idx, bd_ids in enumerate(r_action_null_values_bd_ids):
        print(f"{bd_ids} is R action type and must have a value in {xls.sheet_names[i]}")

BD_00260025 is R action type and must have a value in Sheet1
XYZ is R action type and must have a value in Sheet1
BD_00260019 is R action type and must have a value in Sheet1


In [28]:
# 8.  Weighted Node ID should follow the correct format i.e. BD_XXXXXXXX or C_XXXXXXXX, or R_XXXXXXXX
li = [df['BD_ID'].str.startswith(i, na=False) for i in ["BD_","C_","R_"]]
if not np.sum([np.isnan(i) for i in np.column_stack((li[0],li[1],li[2])).any(axis=1)]) == 0:
    temp = pd.Series(np.column_stack((li[0],li[1],li[2])).any(axis=1))
    incorrect_bd_ids = df.iloc[temp[temp.isna()].index]["BD_ID"]
    for idx, bd_id in enumerate(incorrect_bd_ids):
        validation_errors.append(f"{bd_id} in {xls.sheet_names[i]} doesn't follow the correct format. ")

In [89]:
if not df.__len__() - np.sum(np.column_stack((li[0],li[1],li[2])).any(axis=1)) == 0:
    print("There are some errors.")

There are some errors


In [96]:
incorrect = df.iloc[temp[temp==False].index]["BD_ID"]

In [97]:
for i, bd_id in enumerate(incorrect):
    print(f"{i}:{bd_id}")

0:XYZ
1:2002
2:2


In [71]:
li = [df['BD_ID'].str.startswith(i, na=False) for i in ["BD_","C_","R_"]]
if not df.__len__() - np.sum(np.column_stack((li[0],li[1],li[2])).any(axis=1)) == 0:
    temp = pd.Series(np.column_stack((li[0],li[1],li[2])).any(axis=1))
    incorrect_bd_ids = df.iloc[temp[temp == False].index]["BD_ID"]
    for idx, bd_id in enumerate(incorrect_bd_ids):
        validation_errors.append(f"{bd_id} in {xls.sheet_names[i]} doesn't follow the correct format. ")

In [72]:
validation_errors

['Weighted Node Id against BD_00260025 for Sheet1 is blank.',
 'Weighted Node Id against BD_00260032 for Sheet1 is blank.',
 'Weighted Node Id against R_00260004 for Sheet1 is blank.',
 'Weighted Node Id against R_00260005 for Sheet1 is blank.',
 'Weighted Node Id against R_00260001 for Sheet1 is blank.',
 'Weighted Node Id against R_00260003 for Sheet1 is blank.',
 'Weighted Node Id against R_00260002 for Sheet1 is blank.',
 'Weighted Node Id against 2002 for Sheet1 is blank.',
 'Weighted Node Id against 2 for Sheet1 is blank.',
 'BD_00260025 in Sheet1 have Action Type other than I, R or W.',
 'Action Type for BD_00260025 in Sheet1 cannot be blank. ',
 "2002 in Sheet1 doesn't follow the correct format. ",
 "2 in Sheet1 doesn't follow the correct format. ",
 "2002 in Sheet1 doesn't follow the correct format. ",
 "2 in Sheet1 doesn't follow the correct format. ",
 "XYZ in Sheet1 doesn't follow the correct format. ",
 "2002 in Sheet1 doesn't follow the correct format. ",
 "2 in Sheet1 do

In [348]:
np.sum([np.isnan(i) for i in np.column_stack((li[0],li[1],li[2])).any(axis=1)])

2

In [354]:
li

[0        True
 1        True
 2        True
 3        True
 4        True
 5        True
 6        True
 7        True
 8        True
 9        True
 10       True
 11       True
 12       True
 13       True
 14       True
 15       True
 16       True
 17       True
 18       True
 19       True
 20       True
 21       True
 22       True
 23       True
 24       True
 25       True
 26       True
 27       True
 28       True
 29       True
         ...  
 1626    False
 1627    False
 1628    False
 1629    False
 1630    False
 1631    False
 1632    False
 1633    False
 1634    False
 1635    False
 1636    False
 1637    False
 1638    False
 1639    False
 1640    False
 1641    False
 1642    False
 1643    False
 1644    False
 1645    False
 1646    False
 1647    False
 1648    False
 1649    False
 1650    False
 1651    False
 1652    False
 1653    False
 1654      NaN
 1655      NaN
 Name: BD_ID, Length: 1656, dtype: object, 0       False
 1       False
 2       Fals

In [276]:
np.unique(np.sum([df['BD_ID'].str.startswith(i) for i in ["BD_","C_","R_"]],axis=0,dtype=np.bool))

array([ True])

In [314]:
li = [df['BD_ID'].str.startswith(i) for i in ["BD_","C_","R_"]]

temp = pd.Series(np.column_stack((li[0],li[1],li[2])).any(axis=1))
df.iloc[temp[temp.isna()].index]["BD_ID"]

1654    2002
1655       2
Name: BD_ID, dtype: object

In [255]:
# a, b, c = li[0], li[1], li[2]

In [ ]:
[any(tup) for tup in zip(a,b,c)][-1]

In [208]:
# import re
# pattern = ["BD_","C_","R_"]
# for j in pattern:
#     j = str(j)
#     print([i for i in df["BD_ID"] if re.match(j,i)])

In [ ]:
[i for i in df["BD_ID"] if re.match(["BD_",i)]

In [177]:
df[df["Action Type"]=='W']['Weighted Node ID'].str.contains(' ').__len__()

465

In [198]:
if not df["Action Type"].apply(lambda x: True if x not in ["I","R","W"] else False).sum() == 0:
    unknown_action_type = df[-df["Action Type"].isin(["I","R","W"])]
    unknown_action_bd_ids = unknown_action_type.xs("BD_ID",axis=1).unique()

In [184]:
#[df.parse(i,header=None).query("3 == 'W'") for i in range(len(df.sheet_names))]

In [27]:
count = a.count(a[0])

In [29]:
count==len(df.sheet_names)

True

### Using set index 

In [47]:
df1 = df[df['Values']!=0]
df1 = df1.reset_index(drop=True)
keys = list(df1.columns.values)
i1 = df.set_index(keys).index
i2 = df1.set_index(keys).index
df[i1.isin(i2)]

,BD_ID,BD_NAME,CURRENCY,Action Type,Weighted Node ID,Year,Qtr,Month,Values
1,BD_00260025,Gains / losses volume - Adjustment,MXN,W,,2020,1,2,5.00
2,BD_00260025,Gains / losses volume - Adjustment,MXN,W,,2020,1,3,-5.30
39,BD_00260025,Gains / losses volume - Adjustment,USD,I,,2020,2,4,0.33


In [75]:
df1 = df[df['Values']!=0]
df1 = df1.reset_index(drop=True)
df1.drop(df.columns[2:],axis=1,inplace=True)
keys = list(df.columns.intersection(df1.columns).values)

In [76]:
i1 = df.set_index(keys).index
i2 = df1.set_index(keys).index

In [102]:
df[i1.isin(i2)]

,BD_ID,BD_NAME,CURRENCY,Action Type,Weighted Node ID,Year,Qtr,Month,Values
0,BD_00260025,Gains / losses volume - Adjustment,MXN,W,,2020,1,1,0.0
1,BD_00260025,Gains / losses volume - Adjustment,MXN,W,,2020,1,2,5.0
2,BD_00260025,Gains / losses volume - Adjustment,MXN,W,,2020,1,3,-5.3
3,BD_00260025,Gains / losses volume - Adjustment,MXN,W,,2020,2,4,0.0
4,BD_00260025,Gains / losses volume - Adjustment,MXN,W,,2020,2,5,0.0
5,BD_00260025,Gains / losses volume - Adjustment,MXN,W,,2020,2,6,0.0
6,BD_00260025,Gains / losses volume - Adjustment,MXN,W,,2020,3,7,0.0
7,BD_00260025,Gains / losses volume - Adjustment,MXN,W,,2020,3,8,0.0
8,BD_00260025,Gains / losses volume - Adjustment,MXN,W,,2020,3,9,0.0
9,BD_00260025,Gains / losses volume - Adjustment,MXN,W,,2020,4,10,0.0


### Using index

In [84]:
df1 = df[df["Values"]!=0]

In [87]:
df[df.index.isin(df1.index)].reset_index(drop=True)

,BD_ID,BD_NAME,CURRENCY,Action Type,Weighted Node ID,Year,Qtr,Month,Values
0,BD_00260025,Gains / losses volume - Adjustment,MXN,W,,2020,1,2,5.00
1,BD_00260025,Gains / losses volume - Adjustment,MXN,W,,2020,1,3,-5.30
2,BD_00260025,Gains / losses volume - Adjustment,USD,I,,2020,2,4,0.33


### using mask

In [92]:
df1 = df[df["Values"]!=0].reset_index(drop=True)

In [93]:
df1

,BD_ID,BD_NAME,CURRENCY,Action Type,Weighted Node ID,Year,Qtr,Month,Values
0,BD_00260025,Gains / losses volume - Adjustment,MXN,W,,2020,1,2,5.00
1,BD_00260025,Gains / losses volume - Adjustment,MXN,W,,2020,1,3,-5.30
2,BD_00260025,Gains / losses volume - Adjustment,USD,I,,2020,2,4,0.33


In [99]:
mask = df.BD_ID.isin(df1.BD_ID)
df2 = df[~mask]

In [100]:
df2.shape

(1584, 9)

In [120]:
df[list(df.columns.symmetric_difference(df1.columns[2:]))].head(2)

,BD_ID,BD_NAME
0,BD_00260025,Gains / losses volume - Adjustment
1,BD_00260025,Gains / losses volume - Adjustment


In [126]:
x = pd.DataFrame({"BD_ID":"R_00100016","BD_NAME":"Branches Regions - Average balance - DDA IB MXN - Cuenta maestra","CATEGORY":"Deposit customer balance","PERIOD_YEAR":2019,"PERIOD_MO":1,"VALUE":2500.0,"CURRENCY":"MXN"},index=[0])

In [129]:
y = pd.DataFrame({"BD_TREE_ID":"Branches_L8","PLAN_EXERCISE_ID":"2019_OPERATING_PLAN","BD_ID":"BD_00100002","BALANCE_TYPE":"SUM","PERIOD_YEAR":2016,"PERIOD_QTR":1,"PERIOD_MO":1,"CURRENCY":"MXN","VALUE":6208.3271339346,"SCENARIO_NAME":"ACTUALS","SCENARIO_TYPE":"ACTUALS"},index=[0])

In [130]:
y

,BD_TREE_ID,PLAN_EXERCISE_ID,BD_ID,BALANCE_TYPE,PERIOD_YEAR,PERIOD_QTR,PERIOD_MO,CURRENCY,VALUE,SCENARIO_NAME,SCENARIO_TYPE
0,Branches_L8,2019_OPERATING_PLAN,BD_00100002,SUM,2016,1,1,MXN,6208.327134,ACTUALS,ACTUALS


In [131]:
x

,BD_ID,BD_NAME,CATEGORY,PERIOD_YEAR,PERIOD_MO,VALUE,CURRENCY
0,R_00100016,Branches Regions - Average balance - DDA IB MX...,Deposit customer balance,2019,1,2500.0,MXN


In [133]:
keys = list(x.columns.intersection(y.columns).values)

In [135]:
keys

['BD_ID', 'PERIOD_YEAR', 'PERIOD_MO', 'VALUE', 'CURRENCY']

In [136]:
i1 = x.set_index(keys).index
i2 = y.set_index(keys).index

In [138]:
x[~i1.isin(i2)]

,BD_ID,BD_NAME,CATEGORY,PERIOD_YEAR,PERIOD_MO,VALUE,CURRENCY
0,R_00100016,Branches Regions - Average balance - DDA IB MX...,Deposit customer balance,2019,1,2500.0,MXN
